<center>
<h1> Projet INF8225 :
<h2> Classification de sentiment : LSTM / BiLSTM / google algorithm : BERT
<h4> Patrick Iversenc (2162564)
<h4> Mirado Rabenasolo

    

<h2> Imports :

In [ ]:
!pip install pyspellchecker
!pip install spacy

In [ ]:
!pip install wordcloud

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import spacy
import os
for dirname, _, filenames in os.walk('/archives'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import nltk
from sklearn.feature_selection import SelectKBest, chi2
from nltk.stem.porter import *
import re
from sklearn.pipeline import Pipeline

In [ ]:
import seaborn as sn
from wordcloud import WordCloud
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

<h2> Data Import

In [ ]:
!ls

In [ ]:
# On a un problème d'encoding de la data, nous sommes obligé de la rencodé en latin
# On enlève les lignes que nous ne voulons pas
# On drop les lignes contenant des NaN
full_data = pd.read_csv('../input/data-groups/full_data_cleaned2.csv').dropna()

# On renomme les colones avec nos termes pour être plus à l'aise
columns_names = list(full_data)
full_data.rename(columns={columns_names[0]:"label",
                        columns_names[1]:"text"}, inplace= True)

<h2> Data Visualisation

In [ ]:
full_data.head()

In [ ]:
full_data.describe()

In [ ]:
groups = full_data.groupby('label').count()
list_labels = [0 for i in range(5)]
for index, elt in groups.iterrows():
    list_labels[index]=elt['text']

In [ ]:
plt.bar([0,1,2,3,4],list_labels,align='center',width = 0.5)
plt.show()

##### 

<h2> Preprocessing

In [ ]:
X_stop_words = full_data.sample(n=10000)

In [ ]:
X_stop_words_pos = X_stop_words[X_stop_words['label']==4]
X_stop_words_neg = X_stop_words[X_stop_words['label']==0]

In [ ]:
X_stop_words_pos.describe()

In [ ]:
X_stop_words_neg.describe()

# Preprocessing

<h4> Nous allons créer un pipeline afin de mettre sous le bon format les tweets grâce à la bibliothèque Spacy : <br>
     1. Passer toutes les lettres en lowercase <br>
     2. Enlever la ponctuation<br>
    3. Enlever les mots qui n'apporte rien à la sémentique du tweet <br>
    4. Changer les mots par leur représentant de signification : 'running' -> 'run'
 

In [ ]:
!python3 -m spacy download en_core_web_sm

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def extract_tokens(text):
    """Extract tokens and metadata from individual spaCy doc."""
    doc = nlp(text)
    tokens_return = []
    for i in doc:
        condition = not i.is_punct and not i.is_space and not i.is_stop
        condition2 = not i.is_stop
        #print("here we are looking at {} and the condition {} {}".format(i,condition, condition2))
        if condition:
            #print("we are going from {} to {}".format(j.text,j.lemma_))
            #print(j.lemma_)
            tokens_return.append(i.lemma_)
    return ' '.join(tokens_return)

In [ ]:
def extract_tokens_plus_meta(doc):
    """Extract tokens and metadata from individual spaCy doc."""
    tokens_return = []
    for j in doc:
        for i in j:
            condition = not i.is_punct and not i.is_space 
            condition2 = not i.is_stop
            #print("here we are looking at {} and the condition {} {}".format(i,condition, condition2))
            if condition:
                #print("we are going from {} to {}".format(j.text,j.lemma_))
                #print(j.lemma_)
                tokens_return.append(i.lemma_)
    return tokens_return

In [ ]:
doc_pos = nlp.pipe(X_stop_words_pos.text)
doc_neg = nlp.pipe(X_stop_words_neg.text)

In [ ]:
nb_tokens_pos = []
list_mots = []
nb_pos_dict = {}
tweets_pos=extract_tokens_plus_meta(doc_pos)
#tweets_neg= extract_tokens_plus_meta(doc_neg) 
for mot in tweets_pos:
    if(mot not in list_mots):
        list_mots.append(mot)
        count = tweets_pos.count(mot)
        nb_pos_dict[mot] = count
        nb_tokens_pos.append([mot,count])
    

In [ ]:
nb_tokens_neg = []
tweets_neg=extract_tokens_plus_meta(doc_neg)
#tweets_neg= extract_tokens_plus_meta(doc_neg) 
list_mots2 = []
nb_neg_dict = {}
for mot in tweets_neg:
    if(mot not in list_mots2):
        count = tweets_neg.count(mot)
        list_mots2.append(mot)
        nb_neg_dict[mot] = count
        nb_tokens_neg.append([mot,count])

In [ ]:
#nb_neg_dict['not']

In [ ]:
nb_tokens_pos.sort(key=lambda x: x[1], reverse=True)
nb_tokens_neg.sort(key=lambda x: x[1], reverse=True)
word_tokens_pos = [x[0] for x in nb_tokens_pos[:500]]
word_tokens_neg = [x[0] for x in nb_tokens_neg[:500]]

In [ ]:
def condition(mot,seuille):
    if(not mot in list_mots2):
        return True
    if(not mot in list_mots):
        return True
    if(nb_neg_dict[mot]/nb_pos_dict[mot]>seuille or nb_pos_dict[mot]/nb_neg_dict[mot]> seuille):
        return True

In [ ]:
not_stop_words = []
def creation_stop_words(pos, neg,seuille1,seuille2):
    for mot in word_tokens_pos[:seuille1]:
        if condition(mot,seuille2):
            not_stop_words.append(mot)
    for mot in word_tokens_neg[:seuille1]:
        if condition(mot,seuille2):
            not_stop_words.append(mot)
    

In [ ]:
creation_stop_words(word_tokens_pos,word_tokens_neg,300,2)

In [ ]:
for word in not_stop_words:
    nlp.vocab[word].is_stop=False
    try:
        #print(word)
        nlp.Defaults.stop_words.remove(word)
    except:
        print(word + ' is not in stop words')

In [ ]:
nlp.vocab['not'].is_stop

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
import matplotlib.pyplot as plt

In [ ]:
# We clean the Dataset
#full_data['cleaned_text'] = full_data['text'].apply(extract_tokens)


In [ ]:
# We save the datasets that are cleaned
#full_data.to_csv('./full_data_cleaned.csv')

In [ ]:
def mapping_labels(number):
    if(number==0):
        return 0
    else:
        return 1

In [ ]:
from sklearn.model_selection import train_test_split
full_data = pd.read_csv('../input/data-groups/full_data_cleaned2.csv')
full_data = full_data.dropna()


# Creation d'un Dataset à 5 catégories

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

In [ ]:
sen = SentimentIntensityAnalyzer()
sen.polarity_scores(test_text)

In [ ]:
def map_groups(dict_positivity):
    if(dict_positivity['neg']>0.7):
        return -2
    elif(dict_positivity['pos']>0.7):
        return 2
    elif(dict_positivity['neu']>.7):
        return 0
    elif(dict_positivity['pos']>dict_positivity['neu']+dict_positivity['neg']):
        return 1
    elif(dict_positivity['neg']>dict_positivity['neg']+dict_positivity['pos']):
        return -1
    else:
        return 0

In [ ]:
def mapping_text_to_group(text):
    score = sen.polarity_scores(text)
    return map_groups(score)

In [ ]:
full_data['group5'] = full_data['cleaned_text'].apply(mapping_text_to_group)

<h2> TF-IDF model

In [ ]:
full_data = pd.read_csv('../input/data-groups/full_data_cleaned2.csv')
X_train, X_val, y_train, y_val = train_test_split(full_data['cleaned_text'].values, full_data.label, test_size=0.25)

In [ ]:
financial_data = pd.read_csv('../input/data-groups/financial_data_2_cleaned.csv')
X_train_finance, X_val_finance, y_train_finance, y_val_finance = train_test_split(financial_data['cleaned_text'].values, financial_data.target, test_size=0.25)

In [ ]:
data_group_3 = pd.read_csv('../input/data-groups/eurecom_data.csv').dropna()
X_train3, X_val3, y_train3, y_val3 = train_test_split(data_group_3['cleaned_text'].values, data_group_3.target, test_size=0.25)

In [ ]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                      ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l2', max_iter=20000, dual=False))])
model = pipeline.fit(X_train, y_train)

In [ ]:
pipeline2 = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                      ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l2', max_iter=20000, dual=False))])
model2 = pipeline2.fit(X_train3, y_train3)

In [ ]:
pipeline3 = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                      ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l2', max_iter=20000, dual=False))])
model3 = pipeline3.fit(X_train_finance, y_train_finance)

# Results TFIDF

In [ ]:
vectorizer = model2.named_steps['vect']
chi = model2.named_steps['chi']
clf = model2.named_steps['clf']

feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

target_names = ['-1', '0', '1']
print("top 10 keywords per class:")
important_words = {}
for i, label in enumerate(target_names):
    top = np.argsort(clf2.coef_[i])[-30:]
    important_words[i] = " ".join(feature_names[top])
    print("%s: %s" % (label, " ".join(feature_names[top])))"""

In [ ]:
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                min_font_size = 10).generate(important_words[0])
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()

In [ ]:
from sklearn.metrics import fbeta_score
print("Accuracy score SVC: " + str(model.score(X_val, y_val)))
val_predict = model.predict(X_val)
y_val_array =np.array(y_val)

In [ ]:
confusion_mtx = confusion_matrix(y_val_array,val_predict)
ax = plt.axes()
sn.heatmap(confusion_mtx, annot=True,annot_kws={"size": 15}, cmap="Reds", ax = ax,xticklabels=[-1,0,1],yticklabels=[-1,0,1])
#ax.set_title('Validation Accuracy first Try', size=14)
plt.show()
print(val_predict-y_val_array)
print("Distance between our target and prediction: " + str(np.linalg.norm(val_predict-y_val_array)))

In [ ]:
from sklearn.metrics import fbeta_score
print("Accuracy score SVC: " + str(model2.score(X_train3, y_train3)))
val_predict3 = model2.predict(X_val3)
y_val_array3 =np.array(y_train_finance)
print('F1 score with SVC : ' + str(fbeta_score(y_val_array3,val_predict3,1,average='macro')))

In [ ]:
confusion_mtx = confusion_matrix(y_val_array3,val_predict3)
ax = plt.axes()
sn.heatmap(confusion_mtx, annot=True,annot_kws={"size": 15}, cmap="Reds", ax = ax,xticklabels=[-1,0,1],yticklabels=[-1,0,1])
#ax.set_title('Validation Accuracy first Try', size=14)
plt.show()
print(val_predict-y_val_array)
print("Distance between our target and prediction: " + str(np.linalg.norm(val_predict-y_val_array)))

In [ ]:
from sklearn.metrics import fbeta_score
print("Accuracy score SVC: " + str(model3.score(X_val_finance, y_val_finance)))
val_predict3 = model3.predict(X_val_finance)
y_val_array3 =np.array(y_val_finance)
print('F1 score with SVC : ' + str(fbeta_score(y_val_array3,val_predict3,1,average='macro')))

In [ ]:
confusion_mtx = confusion_matrix(y_val_array3,val_predict3)
ax = plt.axes()
sn.heatmap(confusion_mtx, annot=True,annot_kws={"size": 15}, cmap="Reds", ax = ax,xticklabels=[-1,0,1],yticklabels=[-1,0,1])
#ax.set_title('Validation Accuracy first Try', size=14)
plt.show()
print(val_predict-y_val_array)
print("Distance between our target and prediction: " + str(np.linalg.norm(val_predict-y_val_array)))

# BERT

In [ ]:
! pip install transformers==3

In [ ]:


from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from collections import defaultdict
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [ ]:
from sklearn.model_selection import train_test_split
full_data = pd.read_csv('../input/data-groups/full_data_cleaned2.csv')
full_data = full_data.dropna()
full_data = full_data.sample(10000)
from sklearn.model_selection import train_test_split
Data_train, Data_val = train_test_split(full_data, test_size=0.25)

In [ ]:
mapping_conversion={4:1,0:0}
Data_val['target']=Data_val['label'].map(mapping_conversion)
Data_train['target']=Data_train['label'].map(mapping_conversion)

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
len(X_train)

In [ ]:
import seaborn as sns
from torch.utils.data import Dataset, DataLoader

In [ ]:
Data_train

In [ ]:
token_lens = []
for txt in Data_train.cleaned_text :
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))
    

plt.figure(figsize=(16,8))    
sns.distplot(token_lens)
Max_len = max(token_lens)
plt.xlim([0, 256]);
plt.xlabel('Token count');
print("Max token_lens : ", max(token_lens))

In [ ]:
class SentimentClassifier(nn.Module):
    
  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
model = SentimentClassifier(2)

In [ ]:
class GPReviewDataset(Dataset):
  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.reviews)
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
X_train

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df.cleaned_text.to_numpy(),
    targets=df.target.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )
BATCH_SIZE = 16
train_data_loader = create_data_loader(Data_train, tokenizer, Max_len, BATCH_SIZE)
val_data_loader = create_data_loader(Data_val, tokenizer, Max_len, BATCH_SIZE)

In [ ]:
EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=3e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
    device,
  scheduler,
  n_examples
):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, targets)
      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
istory = defaultdict(list)
best_accuracy = 0
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(Data_train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')
  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn,
    device,
    len(Data_val)
  )
  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc